In [1]:
import os
import requests

import geopandas as gpd
import geojsonio

import pandas as pd
from pandas.io.json import json_normalize

import folium
from folium import plugins

## Schema of data

## Getting data

In [2]:
# Option 1 is preferred: live data fed straight in
r = requests.get("https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_day.geojson")
data = r.json()

In [3]:
# Option 2 - load a csv that was downloaded and saved before (used for the getting the folium map up intially)
# data = pd.read_csv('~/Code/Pybraries/folium/quakes/data/2.5_month.csv')

In [4]:
#data

## Parsing the data

In [5]:
quake_data = pd.json_normalize(data)
quake_data

,type,features,bbox,metadata.generated,metadata.url,metadata.title,metadata.status,metadata.api,metadata.count
0,FeatureCollection,"[{'type': 'Feature', 'properties': {'mag': 2.7...","[-154.0468, -41.7564, -0.77, 179.7463, 73.173,...",1606150049000,https://earthquake.usgs.gov/earthquakes/feed/v...,"USGS Magnitude 2.5+ Earthquakes, Past Day",200,1.10.3,31


In [6]:
quakes = pd.json_normalize(data=data['features'], meta=['mag','place','time','updated','tz','url','detail','felt','cdi','mmi','alert','status','tsunami','sig','net','code','ids','sources','types','nst','dmin','rms','gap','magType'])

Ditch the columns I don't need

In [7]:
quakes.columns = quakes.columns.map(lambda x: x.split(".")[-1])

quakes.drop(['type', 'updated', 'tz', 'url', 'detail', 'felt', 'cdi', 'mmi', 'alert', 'status', 'net', 
                     'code', 'sources', 'types', 'nst', 'dmin', 'rms', 'gap', 'magType' ], axis=1, inplace=True)

Breakout the coords column

In [9]:
quakes['longitude'] = quakes.coordinates.str[0]
quakes['latitude'] = quakes.coordinates.str[1]
quakes['depth'] = quakes.coordinates.str[2]

quakes.drop(['coordinates', 'title'], axis=1, inplace=True)


Figure out how to get time to display

In [10]:
# quakes['datetime'] = pd.to_datetime(['time'], unit='s')
# quakes['datetime'] = pd.to_datetime(quakes["time"], unit='s')

quakes['datetime'] = pd.to_datetime(quakes['time'], infer_datetime_format=True) 
quakes.drop(['time'], axis=1, inplace=True)

In [16]:
quakes.round({'longitude': 5, 'latitude': 5, 'depth': 0})
quakes.head(2)

,id,mag,place,tsunami,sig,ids,longitude,latitude,depth,datetime
0,us6000crm4,2.7,"24 km S of Weston, Colorado",0,112,",us6000crm4,",-104.8953,36.9124,5.0,1970-01-01 00:26:46.139358799
1,us6000crma,4.4,"80 km NNE of Lobujya, Nepal",0,298,",us6000crma,",87.2014,28.5901,10.0,1970-01-01 00:26:46.138905289


In [22]:
m = folium.Map(
    location=[0, 0],
    tiles='cartodbpositron',
    zoom_start=1.5
)



In [14]:
for i, row in quakes.iterrows():
    folium.CircleMarker((row.latitude, row.longitude), radius=row.mag*2.2, 
                        color='red', weight = 0, opacity =.4, fill=True, fill_color='orange', fill_opacity=.3, 
                        popup=[f"Time:{row.datetime},\n Mag: {row.mag},\n Depth: {row.depth} km"]).add_to(m)
    

In [32]:
boundaries = ('data/PB2002_boundaries.json')
line_style = {'color': '#FF3333', 'weight': 1, 'opacity':row.mag*1.4}


folium.GeoJson(
    boundaries,
    name='major fault lines',
    style_function=lambda x:line_style,
    smooth_factor=4.0,
      
).add_to(m)
m